In [1]:
import os
import requests
import json
import openai
import pickle as pkl
import requests
import json
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from time import sleep
from tqdm.auto import tqdm
import numpy as np
import ast
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from sklearn.manifold import TSNE
import plotly.express as px


/Users/haskar140/anaconda3/envs/KG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('azure-configuration.json') as inputfile:
    azureconfig = json.load(inputfile)
openai.api_key = azureconfig['key'] 
openai.api_base = azureconfig['endpoint'] 
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

deployment_name= azureconfig['deployment_name']

response = openai.Embedding.create(
    input="Your text string goes here",
    engine=deployment_name
)
embeddings = response['data'][0]['embedding']
print(len(embeddings))

1536


In [3]:
df_IAB=pd.read_csv('data/IAB_Evaluation.csv')

In [4]:
df_MAF=pd.read_csv('data/MAF_Evaluation.csv')

In [5]:
def clean(item):
    item=item.replace("_", " ")
    item=item.lower()
    return item

In [6]:
df_MAF['item'].apply(clean)

0               crane
1          disk brake
2        auto factory
3                 cab
4          car mirror
            ...      
1872          'stole'
1873     'stone wall'
1874    cutting board
1875     electric ray
1876       infant cry
Name: item, Length: 1877, dtype: object

In [7]:
iab_labels=df_IAB['IAB_Label'].to_list()

In [8]:
maf=df_MAF['item'].to_list()

In [11]:
len(maf)

1877

In [9]:
call=iab_labels+maf

In [10]:
print(call[0:160])

['Hair Care', 'Shaving', 'Motorcycles', "Women's Formal Wear", "Women's Clothing", 'Retail Industry', 'Comedy Events', 'Household Supplies', 'Home Security', 'Polar Travel', 'Auto Parts', 'Awards Shows', 'Business and Finance', 'Barbecues and Grilling', 'Grocery Shopping', 'Auto Body Styles', 'Cooking', "Women's Handbags and Wallets", 'Cruises', 'Coupe', 'World Cuisines', 'Landscaping', 'Commercial Trucks', 'Video Gaming', 'Healthy Living', "Men's Formal Wear", 'Dating', 'Birthday', 'Wedding', 'Museums & Galleries', 'Fitness and Exercise', 'Education industry', 'Tablets and E-readers', 'Retail Property', 'Beach Travel', 'Outdoor Activities', 'Hotels and Motels', 'Space and Astronomy', 'Holiday Shopping', 'Concerts & Music Events', 'Auto Type', 'Entertainment Industry', 'Travel Locations', 'Minivan', 'Smartphones', 'Luxury Cars', 'Travel', 'Perfume and Fragrance', 'Alcoholic Beverages', 'Prom', 'Parks & Nature', 'Science', 'Rail Travel', "Women's Jewelry and Watches", 'Convertible', 'Ca

In [418]:
total_embeddings=[]

In [420]:
def create_embeddings(item,total_embeddings):
    deployment_name= azureconfig['deployment_name']

    response = openai.Embedding.create(
        input=item,
        engine=deployment_name
    )
    embeddings = response['data'][0]['embedding']
    total_embeddings.append(embeddings)

In [422]:
for items in tqdm(call):
    try:
        items=items.replace("&", 'and')
    except:
        pass

#only for Chat GPT version
    # if items in scores.keys():
    #     # print('here')
    #     key = list(scores[items].keys())[0]
    #     create_embeddings(key,total_embeddings)
    #     continue

    create_embeddings(items,total_embeddings)

 22%|██▏       | 454/2037 [00:58<03:24,  7.73it/s]


ServiceUnavailableError: The server is overloaded or not ready yet.

In [ ]:
len(total_embeddings)

2037

In [ ]:
iab_embeddings=total_embeddings[0:160] #Change this if you want to test a new external keyword set

In [ ]:
maf_embeddings=total_embeddings[160:]

In [ ]:
nodes=maf

In [380]:
cos_sim=cosine_similarity(np.array(iab_embeddings[90]).reshape(1,-1),np.array(maf_embeddings[15]).reshape(1,-1))
print(cos_sim)

[[0.77228327]]


In [ ]:
def calculate_precision_recall(true_labels, predictions):
    true_positives = sum(1 for pred in predictions if pred in true_labels)
    false_positives = len(predictions) - true_positives
    false_negatives = len(true_labels) - true_positives
    
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    
    return precision, recall

In [ ]:
def calculate_f1_score(precision, recall):
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [ ]:
with open("data/problematic.pkl", 'rb') as f:
    problematic=pkl.load(f)

In [ ]:
IAB_Candidates=df_MAF['IAB_Candidates'].to_list()

In [ ]:
with open("data/child_parent_dic.pkl", 'rb') as f:
    child_parent_dic=pkl.load(f)

In [ ]:
top_n_labels_new=[]
top_n_cosines_new=[]
top_n_nodes=[]
top_n_node_ids=[]
top_n_IAB_Candidates=[]
top_n_H_scores=[]
top_n_labels=[]
top_n_cosines=[]

tsne_embeddings=[]



for i in tqdm(range(len(maf_embeddings))):
    similarities=[]
    for j in range(len(iab_embeddings)):
        cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
        similarities.append([call[160+i],call[j],cos_sim])
    
    similarities.sort(key=lambda x: x[2], reverse=True)


    #threshold logic needed
    #print(similarities[0:10])
    #similarities=similarities[0:20]
    
    #low initial threshold
    similarities=[x for x in similarities if x[2]>=0.765]
    new_similarities=[]

    # new_similarities=[x for x in similarities if x[2]>=0.8065]

    # if len(new_similarities)==0:
    #     new_similarities.append(['No_pred','No_pred',[[0]]])
        
    

    # #higher threshold for problematic keywords, high threshold for general keywords for accuracy
    for x in similarities:
        if len(new_similarities)>=15 and x[2]<0.82:
            break
        if x[2]>=0.82 and x[1] in problematic:
            new_similarities.append(x)
        elif x[2]>=0.80 and x[1] not in problematic:
            new_similarities.append(x)
        else:
            continue
    
    #adding atleast 1 adword
    if len(new_similarities)==0:
        while len(new_similarities)<1:
            new_similarities.append(similarities[len(new_similarities)])
        #print(new_similarities)

    #print(len(new_similarities))

    #force adding all parent adwords if child is predicted
    new_sim_adwords=[]
    for items in new_similarities:
        new_sim_adwords.append(items[1])

    count=0
    for items in new_similarities:
        if count==5:
            break
        try:
            parent_list=child_parent_dic[items[1]]
            for parent in parent_list:
                if parent not in new_sim_adwords:
                    new_sim_adwords.append(parent)
                    j=call.index(parent)
                    cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
                    if cos_sim>0.765:
                        new_similarities.append([call[160+i],call[j],cos_sim])
        except Exception as e:
            #print(e)
            pass

        count+=1
    

    #new_similarities.sort(key=lambda x: x[2], reverse=True)
     

    #print(len(new_similarities))
        
    temp_iab_labels=[]
    temp_cosines=[]

    count=0
    for items in new_similarities:
        if count>=15 and items[2][0][0]<0.82:
            break
        tsne_embeddings.append(maf_embeddings[i])
        #print(items[1])
        temp_iab_labels.append(items[1])
        temp_cosines.append(items[2][0])
        
        top_n_labels_new.append(items[1])
        top_n_cosines_new.append(items[2][0][0])
        top_n_nodes.append(nodes[i])
        #top_n_node_ids.append(node_id[i])
        top_n_IAB_Candidates.append(IAB_Candidates[i])
        #top_n_H_scores.append(H_Mappings[i])

        count+=1

    
    top_n_labels.append(temp_iab_labels)
    top_n_cosines.append(temp_cosines)

  0%|          | 0/1877 [00:00<?, ?it/s]

100%|██████████| 1877/1877 [02:53<00:00, 10.85it/s]


In [ ]:
len(top_n_labels)

1877

In [ ]:
len(top_n_labels_new)

10829

In [ ]:
MAF_predictions=df_MAF['IAB_Candidates'].to_list()

In [ ]:
candidate_count=0
hit_count=0
precision=0
recall=0
count=0
f1=0

for predictions, labels in zip(MAF_predictions,top_n_labels):
    count+=1
    proper=ast.literal_eval(predictions)
    prec, reca=calculate_precision_recall(predictions, labels)

    f=calculate_f1_score(prec, reca)
    precision+=prec
    recall+=reca
    f1+=f


    candidate_count+=len(proper)
    # for items in labels:
    #     if items in proper:
    #         hit_count+=1

    for items in proper:
        if items in labels:
            hit_count+=1

In [ ]:
print(candidate_count)

5042


In [ ]:
hit_count/candidate_count

0.5358984529948433

In [ ]:
precision/942

0.6011620191191119

In [ ]:
recall/942

0.08785380458952594

In [ ]:
f1/942

0.13941063684867067

In [ ]:
len(top_n_labels_new)

10829

In [ ]:
dic={'item':top_n_nodes,                                             #'IAB_Candidates': top_n_IAB_Candidates,
'predictions': top_n_labels_new, 'cosine_predicted': top_n_cosines_new}

In [ ]:
df_MAF_new=pd.DataFrame.from_dict(dic)

In [ ]:
df_MAF_new.to_csv('results/OpenAI_results_0.80_TSNE_Mapping.csv')

In [ ]:
count=0
for items in tqdm(maf):
    count+=df_MAF_new[df_MAF_new['item']==items].shape[0]

print(count/len(maf))

  2%|▏         | 37/1877 [00:00<00:05, 364.55it/s]

100%|██████████| 1877/1877 [00:02<00:00, 728.26it/s]

8.536494405966968


In [ ]:
counts=[]
for items in tqdm(iab_labels):
    lol=df_MAF_new[df_MAF_new['predictions']==items].shape[0]
    counts.append(lol)

dic={'IAB_Label':iab_labels, 'Count':counts}

df_adwords=pd.DataFrame.from_dict(dic)
df_adwords.to_csv('results/Adwords_Statistics.csv',index=False)

  0%|          | 0/160 [00:00<?, ?it/s]

100%|██████████| 160/160 [00:00<00:00, 731.16it/s]


In [ ]:
iab_labels[counts.index(max(counts))]

'Birthday'

In [ ]:
iab_labels[counts.index(min(counts))]

'Sedan'

In [ ]:
sum(counts)/160

100.14375

In [247]:
#chat GPT Replacement
with open('chatGPT-alternate-name/alternate_names.pkl', 'rb') as f:
    alternate_dic=pkl.load(f)

In [248]:
alternate_dic

defaultdict(list,
            {'Hair Care': ['Haircare',
              'Hair treatment',
              'Hair products',
              'Hair maintenance',
              'Hair solutions'],
             'Shaving': ['Grooming',
              'Trimming',
              'Shorn',
              'Clipping',
              'Razoring'],
             'Motorcycles': ['Bikes',
              'Choppers',
              'Cycles',
              'Hogs',
              'wheelers'],
             "Women's Formal Wear": ["Women's Dressy Attire",
              "Ladies' Evening Wear",
              'Female Formal Clothing',
              "Women's Elegant Outfits",
              "Ladies' Dressier Clothes"],
             "Women's Clothing": ['Ladies fashion',
              'Female attire',
              "Women's apparel",
              "Girls' outfits",
              'Feminine wear'],
             'Retail Industry': ['Commerce',
              'Sales',
              'Shops',
              'Market',
              'Tra

In [249]:
df_MAF_new.head()

,item,predictions,cosine_predicted
0,crane,Birds,0.837830
1,crane,Commercial Trucks,0.827790
2,crane,Cruises,0.826511
3,crane,Pickup Trucks,0.820548
4,crane,Large Animals,0.806308


In [257]:
def create_embeddings_new(item):
    deployment_name= azureconfig['deployment_name']

    response = openai.Embedding.create(
        input=item,
        engine=deployment_name
    )
    embeddings = response['data'][0]['embedding']
    return embeddings

In [367]:
scores={}
for items in tqdm(iab_labels):
    df_orig=df_MAF_new[df_MAF_new['predictions']==items]
    cosine_predicted=sum(df_orig['cosine_predicted'].to_list())/len(df_orig['cosine_predicted'].to_list())
    #print(cosine_predicted)
    #break
    scores[items]={'{}'.format(items):cosine_predicted}
    max_score=cosine_predicted
    for alternates in alternate_dic[items]:
        cos_sims=[]
        alternate_embedding=create_embeddings_new(alternates)
        for keywords in df_orig['item'].to_list():
            i=call.index(keywords)
            item_embedding=total_embeddings[i]
            cos_sims.append(cosine_similarity(np.array(alternate_embedding).reshape(1,-1),np.array(item_embedding).reshape(1,-1)))
        
        cosine_predicted=sum(cos_sims)/len(cos_sims)
        if cosine_predicted>max_score and len(alternates)<len(items):
            max_score=cosine_predicted
            scores[items]={'{}'.format(alternates):cosine_predicted[0][0]}
            if items not in problematic:
                problematic.append(items)
            
    


100%|██████████| 160/160 [03:08<00:00,  1.18s/it]


In [368]:
scores

{'Hair Care': {'Hair Care': 0.8290118891931814},
 'Shaving': {'Shaving': 0.8367466352306965},
 'Motorcycles': {'Motorcycles': 0.8247143667824867},
 "Women's Formal Wear": {"Women's Formal Wear": 0.8198046257508906},
 "Women's Clothing": {"Women's Clothing": 0.8205189799687213},
 'Retail Industry': {'Retail Industry': 0.8275536253308498},
 'Comedy Events': {'Comedy Events': 0.8129531939870166},
 'Household Supplies': {'Household Supplies': 0.8193957579424482},
 'Home Security': {'Home Security': 0.8179680203041682},
 'Polar Travel': {'Polar Travel': 0.819079952087211},
 'Auto Parts': {'Auto Parts': 0.8266459128584055},
 'Awards Shows': {'Awards Shows': 0.8218537807832149},
 'Business and Finance': {'Business and Finance': 0.8013148548164415},
 'Barbecues and Grilling': {'Barbecues and Grilling': 0.8154770590344835},
 'Grocery Shopping': {'Grocery Shopping': 0.8250322940246976},
 'Auto Body Styles': {'Auto Body Styles': 0.804997637402198},
 'Cooking': {'Cooking': 0.8367442921889144},
 "W

In [369]:
original_name=[]
best_name=[]
score=[]
for k,v in scores.items():
    original_name.append(k)
    for kk, vv in v.items():
        best_name.append(kk)
        score.append(vv)

In [370]:
new_dic={'Original_name':original_name, "New_name":best_name, "Average Score":score }

In [371]:
df_new_names=pd.DataFrame.from_dict(new_dic)

In [372]:
df_new_names.to_csv('results/New_IAB_Labels_only_shorter.csv', index=False)

In [121]:
df_tiers=pd.read_csv('data/MAFlabel_to_Adword_mappings_202210_combined_mappings.csv')

In [122]:
df_tiers=df_tiers['adword']

In [123]:
tier1=[]
all_tiers=df_tiers.to_list()
for tier in all_tiers:
    try:
        tier1.append(tier.split('|')[0])
    except:
        tier1.append(tier)

tier1=list(set(tier1))

In [47]:
len(tier1)

26

In [48]:
diff_Tier=['Events and Attractions', 'Fashion Events','Malls & Shopping Centers', 'Museums & Galleries','Outdoor Activities','Parks & Nature','Awards Shows','Personal Celebrations & Life Events','Wedding','Birthday','Graduation','Prom','Religious Events','Theater Venues and Events','Zoos & Aquariums','Bars & Restaurants','Casinos & Gambling','Cinemas and Events','Comedy Events','Concerts & Music Events']

In [49]:
diff_Tier=['Cats'] #,'Flower Shopping','Grocery Shopping','Holiday Shopping','Household Supplies',"Children's Games and Toys"]

In [50]:
dic={'embeddings':tsne_embeddings, 'labels':top_n_labels_new, 'item': top_n_nodes}
df_labels_embeddings=pd.DataFrame.from_dict(dic)

In [51]:
df_labels_embeddings.head

<bound method NDFrame.head of                                               embeddings  \
0      [-0.016414720565080643, -0.027218835428357124,...   
1      [-0.016414720565080643, -0.027218835428357124,...   
2      [-0.016414720565080643, -0.027218835428357124,...   
3      [-0.016414720565080643, -0.027218835428357124,...   
4      [-0.016414720565080643, -0.027218835428357124,...   
...                                                  ...   
12301  [-0.023140832781791687, -0.0161676537245512, -...   
12302  [-0.023140832781791687, -0.0161676537245512, -...   
12303  [-0.023140832781791687, -0.0161676537245512, -...   
12304  [-0.011204354465007782, 0.002425432438030839, ...   
12305  [-0.011204354465007782, 0.002425432438030839, ...   

                         labels          item  
0                         Birds         crane  
1             Commercial Trucks         crane  
2                       Cruises         crane  
3                 Pickup Trucks         crane  
4        

In [52]:
results={}
for label in tqdm(iab_labels):
    mask = df_labels_embeddings['labels'].isin([label])
    df_tier1 = df_labels_embeddings[mask]
    if df_tier1.shape[0]//2<50:
        perp=df_tier1.shape[0]//2
    else:
        perp=50
    to_TSNE_list=df_tier1['embeddings'].to_list()
    to_TSNE_arr=np.array(to_TSNE_list)
    tsne=TSNE(n_components=2, random_state=42, perplexity=perp)
    X_embedded = tsne.fit_transform(to_TSNE_arr)
    results[label]=tsne.kl_divergence_


  0%|          | 0/160 [00:00<?, ?it/s]/Users/haskar140/anaconda3/envs/KG/lib/python3.11/site-packages/threadpoolctl.py:1010: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
100%|██████████| 160/160 [01:02<00:00,  2.56it/s]


In [53]:
count=0
#problematic=[]
new_problematic=[]
for k,v in results.items():
    if k not in problematic and v>1.5:
        print(k,v)
        new_problematic.append(k)

In [54]:
problematic=problematic+new_problematic

In [55]:
problematic

['Shaving',
 'Cooking',
 'Coupe',
 'Dating',
 'Wedding',
 'Minivan',
 'Travel',
 'Prom',
 'Science',
 'Convertible',
 'Camping',
 'SUV',
 'Birds',
 'Sports',
 'Shopping',
 'Beauty',
 'Graduation',
 'Gardening',
 'Bath and Shower',
 'Wellness',
 'Station Wagon',
 'Desktops',
 'Movies',
 'Education',
 'Cats',
 'Cruises',
 'Landscaping',
 'Outdoor Activities',
 'Holiday Shopping',
 'Smartphones',
 'Rail Travel',
 'Reptiles',
 'Home & Garden',
 'Sedan',
 'Laptops',
 'Parenting',
 'Pets',
 'Games and Puzzles',
 'Apartments',
 'Houses',
 'Dining Out',
 'Television',
 'Automotive',
 'Dogs']

In [56]:
mask = df_labels_embeddings['labels'].isin(tier1)
df_tier1 = df_labels_embeddings[mask]

In [57]:
df_tier1.head

<bound method NDFrame.head of                                               embeddings  \
6      [-0.016414720565080643, -0.027218835428357124,...   
7      [-0.016414720565080643, -0.027218835428357124,...   
9      [-0.016414720565080643, -0.027218835428357124,...   
10     [-0.016414720565080643, -0.027218835428357124,...   
12     [-0.016414720565080643, -0.027218835428357124,...   
...                                                  ...   
12297  [-0.017885692417621613, -0.019642207771539688,...   
12298  [-0.017885692417621613, -0.019642207771539688,...   
12302  [-0.023140832781791687, -0.0161676537245512, -...   
12303  [-0.023140832781791687, -0.0161676537245512, -...   
12305  [-0.011204354465007782, 0.002425432438030839, ...   

                         labels           item  
6                          Pets          crane  
7                    Automotive          crane  
9                        Travel          crane  
10                   Automotive          crane  
12  

In [58]:
df_tier1.shape

(4769, 3)

In [59]:
to_TSNE_list=df_tier1['embeddings'].to_list()

In [60]:
to_TSNE_arr=np.array(to_TSNE_list)

In [61]:
if df_tier1.shape[0]//2<50:
    perp=df_tier1.shape[0]//2
else:
    perp=50

In [62]:
tsne=TSNE(n_components=2, verbose=1, random_state=42, perplexity= perp) #df_tier1.shape[0]//2)
X_embedded = tsne.fit_transform(to_TSNE_arr)
tsne.kl_divergence_

[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Indexed 4769 samples in 0.006s...
[t-SNE] Computed neighbors for 4769 samples in 1.015s...
[t-SNE] Computed conditional probabilities for sample 1000 / 4769
[t-SNE] Computed conditional probabilities for sample 2000 / 4769
[t-SNE] Computed conditional probabilities for sample 3000 / 4769
[t-SNE] Computed conditional probabilities for sample 4000 / 4769
[t-SNE] Computed conditional probabilities for sample 4769 / 4769
[t-SNE] Mean sigma: 0.257659
[t-SNE] KL divergence after 250 iterations with early exaggeration: 81.963226
[t-SNE] KL divergence after 1000 iterations: 0.936082


0.9360816478729248

In [63]:
df_embeddings = pd.DataFrame(X_embedded)
df_embeddings = df_embeddings.rename(columns={0:'x',1:'y'})
df_embeddings = df_embeddings.assign(label=df_tier1.labels.values)
df_embeddings = df_embeddings.assign(items=df_tier1.item.values)

In [64]:
df_embeddings.head()

,x,y,label,items
0,-47.119091,-10.815174,Pets,crane
1,-47.635616,-10.732237,Automotive,crane
2,-47.939156,-10.208823,Travel,crane
3,-47.484772,-9.886748,Automotive,crane
4,-46.994915,-10.210792,Pets,crane


In [65]:
fig = px.scatter(
    df_embeddings, x='x', y='y',
    color='label', labels={'color': 'label'},
    hover_data=['items'], title = 'Tier1')
fig.show()

In [66]:
fig.write_image("Tier1.png")

In [67]:
fig.write_html("Tier1.html")
